Downloading Data:
=================

- Client ID: sh-d89408a3-f5db-4c8b-a39d-929f9f299aa9
- Client secret: BqJ60XbDC85suN5gVlqYy6JjmJ5nwdT5

Links:
- https://documentation.dataspace.copernicus.eu/APIs/SentinelHub/Process/Examples/S2L2A.html#true-color
- https://documentation.dataspace.copernicus.eu/notebook-samples/sentinelhub/cloudless_process_api.html



In [2]:
# Print basic timestamp and python version information

import time
import sys

print("Timestamp: ", time.strftime("%Y-%m-%d %H:%M:%S"))
print("Python version: ", sys.version)

Timestamp:  2025-01-27 13:11:40
Python version:  3.12.5 (tags/v3.12.5:ff3bc82, Aug  6 2024, 20:45:27) [MSC v.1940 64 bit (AMD64)]


In [41]:
# Setup the environment with pip installs and imports

installs = ["sentinelhub", "numpy", "pandas", "matplotlib", "requests", "getpass", "scipy"]
for package in installs:
    !pip install {package}

ERROR: Could not find a version that satisfies the requirement getpass (from versions: none)
ERROR: No matching distribution found for getpass


   ---------------------------------------- 0.0/43.6 MB ? eta -:--:--
   ---------------------------------------- 0.3/43.6 MB ? eta -:--:--
    --------------------------------------- 1.0/43.6 MB 3.3 MB/s eta 0:00:13
   -- ------------------------------------- 2.4/43.6 MB 5.0 MB/s eta 0:00:09
   --- ------------------------------------ 3.9/43.6 MB 5.7 MB/s eta 0:00:07
   ----- ---------------------------------- 5.8/43.6 MB 6.5 MB/s eta 0:00:06
   ------ --------------------------------- 7.6/43.6 MB 7.1 MB/s eta 0:00:06
   -------- ------------------------------- 9.4/43.6 MB 7.2 MB/s eta 0:00:05
   ---------- ----------------------------- 11.3/43.6 MB 7.6 MB/s eta 0:00:05
   ------------ --------------------------- 13.1/43.6 MB 7.8 MB/s eta 0:00:04
   ------------- -------------------------- 14.9/43.6 MB 7.9 MB/s eta 0:00:04
   --------------- ------------------------ 17.0/43.6 MB 8.1 MB/s eta 0:00:04
   ----------------- ---------------------- 18.9/43.6 MB 8.2 MB/s eta 0:00:04
   -----

In [3]:


# Utilities
import os
import datetime
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import requests
import getpass

from sentinelhub import (
    SHConfig,
    DataCollection,
    SentinelHubCatalog,
    SentinelHubRequest,
    BBox,
    bbox_to_dimensions,
    CRS,
    MimeType,
    Geometry,
)


In [4]:
# Your client credentials
#Client ID: sh-d89408a3-f5db-4c8b-a39d-929f9f299aa9
#Client secret: BqJ60XbDC85suN5gVlqYy6JjmJ5nwdT5
client_id = 'sh-d89408a3-f5db-4c8b-a39d-929f9f299aa9'
client_secret = 'BqJ60XbDC85suN5gVlqYy6JjmJ5nwdT5'

In [5]:
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session

def SetupOAuth(client_id, client_secret):

    # Create a session
    client = BackendApplicationClient(client_id=client_id)
    oauth = OAuth2Session(client=client)

    # Get token for the session
    token = oauth.fetch_token(token_url='https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token',
                            client_secret=client_secret, include_client_id=True)

    # All requests using this session will have an access token automatically added
    resp = oauth.get("https://sh.dataspace.copernicus.eu/configuration/v1/wms/instances")

    return oauth

oauth = SetupOAuth(client_id, client_secret)


In [6]:
# Create strings to support the query

def CreateBBoxString(minlon, minlat, maxlon, maxlat):
    return f"[{minlon}, {minlat}, {maxlon}, {maxlat}]"

def CreateTimerangeString(date):
    # add buffer around the stated date
    datespace = datetime.timedelta(days=2)
    startdate = date - datespace
    enddate = date + datespace

    # format individual date strings
    from_date = f"{startdate.strftime('%Y-%m-%d')}T00:00:00Z"
    to_date = f"{enddate.strftime('%Y-%m-%d')}T23:59:59Z"

    # return the collated/formatted answer
    return {"from": from_date, "to": to_date}



In [112]:
from PIL import Image
from io import BytesIO

oauth = SetupOAuth(client_id, client_secret)

maxlat = 50.8
minlat = 50.7
minlon = -1.4
maxlon = -1.3
bboxstring = CreateBBoxString(minlon, minlat, maxlon, maxlat)

print(bboxstring)


date = "2024-11-26"
time_range = CreateTimerangeString(datetime.datetime.strptime(date, "%Y-%m-%d"))

evalscript_clean = """
//VERSION=3
function setup() {
  return {
    input: ["B02", "B03", "B04", "SCL"],
    output: { bands: 3 },
  }
}

function evaluatePixel(sample) {
    return [2.5 * sample.B04, 2.5 * sample.B03, 2.5 * sample.B02]
}
"""

evalscript_cloud = """
//VERSION=3
function setup() {
  return {
    input: ["B02", "B03", "B04", "SCL"],
    output: { bands: 3 },
  }
}

function evaluatePixel(sample) {
  if ([8, 9, 10].includes(sample.SCL)) {
    return [1, 0, 1]
  } else {
    return [0, 0, 0]
  }
}
"""

request = {
    "input": {
        "bounds": {
            "properties": {"crs": "http://www.opengis.net/def/crs/EPSG/0/4326"},
            "bbox": [-1.4, 50.7, -1.3, 50.8],
        },
        "data": [
            {
                "type": "sentinel-2-l2a",
                "dataFilter": {"timeRange": time_range}
            }
        ]
    },
    "output": {
        "width":  1024,
        "height": 1024,
        "responses": [
            {
                "identifier": "default",
                "format": {"type": "image/png"}
            }
        ]
    },
    "evalscript": evalscript_clean
}












evalscript_GM = """
//VERSION=3
function setup() {
  return {
    input: ["B02", "B03", "B04"],
    output: { bands: 3 }
  };
}

function evaluatePixel(sample) {
  return [2.5 * sample.B04/10000, 2.5 * sample.B03/10000, 2.5 * sample.B02/10000];
}
"""

request_GM = {
  "input": {
    "bounds": {
      "bbox": [-10, 50, 0, 60]
    },
    "data": [
      {
        "dataFilter": {
          "timeRange": {
            "from": "2023-04-01T00:00:00Z",
            "to": "2023-04-01T23:59:59Z"
          }
        },
        "type": "byoc-5460de54-082e-473a-b6ea-d5cbe3c17cca"

      }
    ]
  },
  "output": {
    "width": 1024,
    "height": 1024,
    "responses": [
      {
        "identifier": "default",
        "format": {
          "type": "image/jpeg"
        }
      }
    ]
  },
  "evalscript": evalscript_GM,
}













url = "https://sh.dataspace.copernicus.eu/api/v1/process"
response = oauth.post(url, json=request_GM)

# Output the result to a file
if response.status_code == 200:
    image = Image.open(BytesIO(response.content))
    image.save("output_image6.png")
else:
    print(f"Request failed with status code {response.status_code}")



[-1.4, 50.7, -1.3, 50.8]


In [39]:
from PIL import Image
from io import BytesIO


def download_GM(outpath, bl_lat, bl_lon, width):

    minlat = bl_lat
    maxlat = bl_lat + width
    minlon = bl_lon
    maxlon = bl_lon + width

    evalscript_GM = """
    //VERSION=3
    function setup() {
        return {
            input: ["B02", "B03", "B04"],
            output: { bands: 3 }
        };
    }

    function evaluatePixel(sample) {
        return [3.5 * sample.B04/10000, 3.5 * sample.B03/10000, 3.5 * sample.B02/10000];
    }
    """

    request_GM = {
    "input": {
        "bounds": {
        "bbox": [minlon, minlat, maxlon, maxlat]
        },
        "data": [
        {
            "dataFilter": {
            "timeRange": {
                "from": "2023-07-01T00:00:00Z",
                "to": "2023-07-01T23:59:59Z"
            }
            },
            "type": "byoc-5460de54-082e-473a-b6ea-d5cbe3c17cca"

        }
        ]
    },
    "output": {
        "width": 1024,
        "height": 1024,
        "responses": [
        {
            "identifier": "default",
            "format": {
                "type": "image/png"
            }
        }
        ]
    },
    "evalscript": evalscript_GM,
    }

    url = "https://sh.dataspace.copernicus.eu/api/v1/process"
    response = oauth.post(url, json=request_GM)

    # Output the result to a file
    if response.status_code == 200:
        image = Image.open(BytesIO(response.content))
        image.save(outpath)
    else:
        print(f"Request failed with status code {response.status_code}")



Design

- We have an "input stripe" of data, with min/max lat long values.
- Pass across the input stripe at a 0.1 degree level.
    - Lower res while testing
- Download cloud image
- Download "clean" unaltered image
    - Fix the clean image exposure


In [ ]:
stripe_min_lat = 50.7
stripe_max_lat = 50.8
stripe_min_lon = -1.4
stripe_max_lon = -1.3

In [7]:
from PIL import Image, ImageEnhance
import numpy as np

def enhance_image_brightness(input_path, output_path, desired_brightness):
    # Load the uploaded image
    image = Image.open(input_path)

    # Convert the image to grayscale and calculate its brightness, excluding black pixels
    grayscale_image = image.convert("L")
    grayscale_array = np.array(grayscale_image)
    non_black_pixels = grayscale_array[grayscale_array > 0]
    brightness = np.mean(non_black_pixels) if non_black_pixels.size > 0 else 0

    # Calculate enhancement factor
    enhancement_factor = desired_brightness / brightness if brightness > 0 else 1

    # Enhance the image brightness
    enhancer = ImageEnhance.Brightness(image)
    enhanced_image = enhancer.enhance(enhancement_factor)

    # Save the enhanced image
    enhanced_image.save(output_path)

# Example usage
#desired_brightness = 100
#input_path = 'C:/Util/GlobeLibrary_MapPrep/SupportingTools/MapDownload/output_image6.png'
#output_path = 'C:/Util/GlobeLibrary_MapPrep/SupportingTools/MapDownload/output_image6_b.png'
#enhance_image_brightness(input_path, output_path, desired_brightness)

def change_absolute_brightness(input_path, output_path, brightness_delta):
    """
    Adjusts the brightness of an image by adding a fixed delta to each pixel.

    Args:
        input_path (str): Path to the input image.
        output_path (str): Path to save the output image.
        brightness_delta (int): The brightness change to apply (positive or negative).
    """
    # Open the image and convert it to NumPy array
    image = Image.open(input_path)
    image_array = np.array(image, dtype=np.int16)  # Use int16 to avoid overflow during adjustment

    # Add brightness delta to all pixels
    image_array += brightness_delta

    # Clip values to the valid range [0, 255] for image data
    image_array = np.clip(image_array, 0, 255).astype(np.uint8)

    # Convert back to image and save
    enhanced_image = Image.fromarray(image_array)
    enhanced_image.save(output_path)



In [8]:
import numpy as np
import cv2

def analyze_image3(input_path, output_path, max_distance=100):

    # Load the image
    image = cv2.imread(input_path)

    # Define magenta color in BGR
    magenta = np.array([255, 0, 255])

    max_value = 255

    # Factor to multiply the distance by to normalize it to the output greyscale range [0, 255]
    max_distance_to_max_value = max_value / max_distance

    # Create an output image initialized to black
    output_image = np.zeros(image.shape[:2], dtype=np.float32)
    output_image.fill(max_value)

    # Iterate through each pixel in the image
    for y in range(image.shape[0]):
        for x in range(image.shape[1]):

            if np.array_equal(image[y, x], magenta):

                # Write a pattern of values into the output image
                for dy in range(-max_distance, max_distance + 1):
                    for dx in range(-max_distance, max_distance + 1):

                        # Find the output pixel coordinates and check its within the image bounds
                        ny, nx = y + dy, x + dx
                        if 0 <= ny < image.shape[0] and 0 <= nx < image.shape[1]:

                            # Calculate the distance between the current pixel and the magenta pixel
                            distance = np.sqrt(dx**2 + dy**2)

                            # Clip the distance to the maximum distance, so all potential values are [0, max_distance]
                            distance = min(distance, max_distance)

                            # Normalize the distance to the range [0, 255]
                            val = distance * max_distance_to_max_value

                            # The image accumulates the minimum distance value to a magenta pixel,
                            # either the current value or the new value
                            output_image[ny, nx] = min(output_image[ny, nx], distance)

    # Normalize the output image to the range [0, 255]
    output_image = (output_image).astype(np.uint8)

    # Save the output image
    cv2.imwrite(output_path, output_image)


In [9]:
import numpy as np
import cv2

def analyze_image_optimized(input_path, output_path, max_distance=100):
    # Load the image
    image = cv2.imread(input_path)

    # Define magenta color in BGR
    magenta = np.array([255, 0, 255])

    # Create a mask of magenta pixels
    mask = np.all(image == magenta, axis=-1).astype(np.uint8)

    # Compute the Euclidean distance transform for the mask
    distances = cv2.distanceTransform(1 - mask, cv2.DIST_L2, 3)

    # Clip distances to max_distance and normalize to [0, 255]
    distances = np.clip(distances, 0, max_distance)
    distances = (distances / max_distance * 255).astype(np.uint8)

    # Save the output image
    cv2.imwrite(output_path, distances)


In [97]:

input_cloud_path  = 'C:/Util/GlobeLibrary_MapPrep/SupportingTools/MapDownload/output_image4.png'
output_cloud_path = 'C:/Util/GlobeLibrary_MapPrep/SupportingTools/MapDownload/output_image4_g.png'

# Example usage
analyze_image_optimized(input_cloud_path, output_cloud_path)

In [10]:
import os

# function to make sure a path exists. If not, create it
def ensure_dir(file_path):
    directory = os.path.dirname(file_path)
    if not os.path.exists(directory):
        os.makedirs(directory)

In [11]:
from PIL import Image, ImageEnhance
import numpy as np
import os

def adjust_image_properties(input_path, output_path, brightness_delta=0, contrast_factor=1.0, saturation_factor=1.0):
    """
    Adjusts the brightness, contrast, and saturation of an image.

    Args:
        input_path (str): Path to the input image.
        output_path (str): Path to save the adjusted image.
        brightness_delta (int): Absolute brightness adjustment. Adds this value to each pixel.
        contrast_factor (float): Contrast adjustment factor. 1.0 = no change, >1.0 = increase, <1.0 = decrease.
        saturation_factor (float): Saturation adjustment factor. 1.0 = no change, >1.0 = increase, <1.0 = decrease.
    """
    # Open the image
    image = Image.open(input_path)

    # Adjust brightness (absolute adjustment)
    if brightness_delta != 0:
        image_array = np.array(image, dtype=np.int16)  # Use int16 to avoid overflow
        image_array += brightness_delta
        image_array = np.clip(image_array, 0, 255).astype(np.uint8)  # Clip to valid range
        image = Image.fromarray(image_array)

    # Adjust contrast
    if contrast_factor != 1.0:
        enhancer = ImageEnhance.Contrast(image)
        image = enhancer.enhance(contrast_factor)

    # Adjust saturation
    if saturation_factor != 1.0:
        enhancer = ImageEnhance.Color(image)
        image = enhancer.enhance(saturation_factor)

    # Save the adjusted image
    image.save(output_path)

# Example usage
# adjust_image_properties(
#     input_path="input_tile.png",
#     output_path="output_tile.png",
#     brightness_delta=30,
#     contrast_factor=1.2,
#     saturation_factor=1.5
# )

# Example usage
# adjust_image_properties(input_path="input_tile.png", output_path="output_tile.png", brightness_delta=30, contrast_factor=1.2, saturation_factor=1.5)


In [45]:
#Path - Code, bl, size



RootDir = "C:/Util/GlobeLibrary_MapPrep/_WIP/exp/MapLib/Lvl1_5x5/EL/"

# join path elements into filename
fullfilepath = os.path.join(RootDir, "Sat_EL_DD.png")



#EL_DD, -50, 165, 5





ensure_dir(RootDir)

oauth = SetupOAuth(client_id, client_secret)

brightness = 50

fullfilepath = os.path.join(RootDir, "Sat_EL_DD.png")
if not os.path.exists(fullfilepath):
    download_GM(fullfilepath, -50, 165, 5)
    enhance_image_brightness(fullfilepath, fullfilepath, brightness)

fullfilepath = os.path.join(RootDir, "Sat_EL_CD.png")
if not os.path.exists(fullfilepath):
    download_GM(fullfilepath, -45, 165, 5)
    enhance_image_brightness(fullfilepath, fullfilepath, brightness)

fullfilepath = os.path.join(RootDir, "Sat_EL_DE.png")
if not os.path.exists(fullfilepath):
    download_GM(fullfilepath, -50, 170, 5)
    enhance_image_brightness(fullfilepath, fullfilepath, brightness)

fullfilepath = os.path.join(RootDir, "Sat_EL_CE.png")
if not os.path.exists(fullfilepath):
    download_GM(fullfilepath, -45, 170, 5)
    enhance_image_brightness(fullfilepath, fullfilepath, brightness)

fullfilepath = os.path.join(RootDir, "Sat_EL_CF.png")
if not os.path.exists(fullfilepath):
    download_GM(fullfilepath, -45, 175, 5)
    enhance_image_brightness(fullfilepath, fullfilepath, brightness)

fullfilepath = os.path.join(RootDir, "Sat_EL_BE.png")
if not os.path.exists(fullfilepath):
    download_GM(fullfilepath, -40, 170, 5)
    enhance_image_brightness(fullfilepath, fullfilepath, brightness)

fullfilepath = os.path.join(RootDir, "Sat_EL_BF.png")
if not os.path.exists(fullfilepath):
    download_GM(fullfilepath, -40, 175, 5)
    enhance_image_brightness(fullfilepath, fullfilepath, brightness)

fullfilepath = os.path.join(RootDir, "Sat_EL_AE.png")
if not os.path.exists(fullfilepath):
    download_GM(fullfilepath, -35, 170, 5)
    enhance_image_brightness(fullfilepath, fullfilepath, brightness)

# -----------------


RootDir = "C:/Util/GlobeLibrary_MapPrep/_WIP/exp/MapLib/Lvl2_1x1/EL_DD/"
ensure_dir(RootDir)

fullfilepath = os.path.join(RootDir, "Sat_EL_DD_CC.png")
if not os.path.exists(fullfilepath):
    download_GM(fullfilepath, -48, 167, 1)
    enhance_image_brightness(fullfilepath, fullfilepath, brightness)


fullfilepath = os.path.join(RootDir, "Sat_EL_DD_CD.png")
if not os.path.exists(fullfilepath):
    download_GM(fullfilepath, -48, 168, 1)
    enhance_image_brightness(fullfilepath, fullfilepath, brightness)

fullfilepath = os.path.join(RootDir, "Sat_EL_DD_BB.png")
if not os.path.exists(fullfilepath):
    download_GM(fullfilepath, -47, 166, 1)
    enhance_image_brightness(fullfilepath, fullfilepath, brightness)

fullfilepath = os.path.join(RootDir, "Sat_EL_DD_BC.png")
if not os.path.exists(fullfilepath):
    download_GM(fullfilepath, -47, 167, 1)
    enhance_image_brightness(fullfilepath, fullfilepath, brightness)

fullfilepath = os.path.join(RootDir, "Sat_EL_DD_BD.png")
if not os.path.exists(fullfilepath):
    download_GM(fullfilepath, -47, 168, 1)
    enhance_image_brightness(fullfilepath, fullfilepath, brightness)

fullfilepath = os.path.join(RootDir, "Sat_EL_DD_BE.png")
if not os.path.exists(fullfilepath):
    download_GM(fullfilepath, -47, 169, 1)
    enhance_image_brightness(fullfilepath, fullfilepath, brightness)

fullfilepath = os.path.join(RootDir, "Sat_EL_DD_AB.png")
if not os.path.exists(fullfilepath):
    download_GM(fullfilepath, -46, 166, 1)
    enhance_image_brightness(fullfilepath, fullfilepath, brightness)

fullfilepath = os.path.join(RootDir, "Sat_EL_DD_AC.png")
if not os.path.exists(fullfilepath):
    download_GM(fullfilepath, -46, 167, 1)
    enhance_image_brightness(fullfilepath, fullfilepath, brightness)

fullfilepath = os.path.join(RootDir, "Sat_EL_DD_AD.png")
if not os.path.exists(fullfilepath):
    download_GM(fullfilepath, -46, 168, 1)
    enhance_image_brightness(fullfilepath, fullfilepath, brightness)

fullfilepath = os.path.join(RootDir, "Sat_EL_DD_AE.png")
if not os.path.exists(fullfilepath):
    download_GM(fullfilepath, -46, 169, 1)
    enhance_image_brightness(fullfilepath, fullfilepath, brightness)



In [12]:
# Data structure for the metadata
image_data_5Deg = [
    {"filename": "Sat_EL_DD.png", "lat": -50, "long": 165, "width": 5},
    {"filename": "Sat_EL_CD.png", "lat": -45, "long": 165, "width": 5},
    {"filename": "Sat_EL_DE.png", "lat": -50, "long": 170, "width": 5},
]

image_data_1Deg = [
    {"localpath": "Lvl2_1x1/EL_DD/", "filename": "Sat_EL_DD_CC.png", "lat": -48, "long": 167, "width": 1},
    {"localpath": "Lvl2_1x1/EL_DD/", "filename": "Sat_EL_DD_CD.png", "lat": -48, "long": 168, "width": 1},
    {"localpath": "Lvl2_1x1/EL_DD/", "filename": "Sat_EL_DD_BB.png", "lat": -47, "long": 166, "width": 1},
    {"localpath": "Lvl2_1x1/EL_DD/", "filename": "Sat_EL_DD_BC.png", "lat": -47, "long": 167, "width": 1},
    {"localpath": "Lvl2_1x1/EL_DD/", "filename": "Sat_EL_DD_BD.png", "lat": -47, "long": 168, "width": 1},
    {"localpath": "Lvl2_1x1/EL_DD/", "filename": "Sat_EL_DD_BE.png", "lat": -47, "long": 169, "width": 1},
    {"localpath": "Lvl2_1x1/EL_DD/", "filename": "Sat_EL_DD_AB.png", "lat": -46, "long": 166, "width": 1},
    {"localpath": "Lvl2_1x1/EL_DD/", "filename": "Sat_EL_DD_AC.png", "lat": -46, "long": 167, "width": 1},
    {"localpath": "Lvl2_1x1/EL_DD/", "filename": "Sat_EL_DD_AD.png", "lat": -46, "long": 168, "width": 1},
    {"localpath": "Lvl2_1x1/EL_DD/", "filename": "Sat_EL_DD_AE.png", "lat": -46, "long": 169, "width": 1},

    {"localpath": "Lvl2_1x1/EL_CD/", "filename": "Sat_EL_CD_EC.png", "lat": -45, "long": 167, "width": 1},
    {"localpath": "Lvl2_1x1/EL_CD/", "filename": "Sat_EL_CD_ED.png", "lat": -45, "long": 168, "width": 1},
    {"localpath": "Lvl2_1x1/EL_CD/", "filename": "Sat_EL_CD_EE.png", "lat": -45, "long": 169, "width": 1},
    {"localpath": "Lvl2_1x1/EL_CD/", "filename": "Sat_EL_CD_DC.png", "lat": -44, "long": 167, "width": 1},
    {"localpath": "Lvl2_1x1/EL_CD/", "filename": "Sat_EL_CD_DD.png", "lat": -44, "long": 168, "width": 1},
    {"localpath": "Lvl2_1x1/EL_CD/", "filename": "Sat_EL_CD_DE.png", "lat": -44, "long": 169, "width": 1},

    {"localpath": "Lvl2_1x1/EL_DE/", "filename": "Sat_EL_DE_AA.png", "lat": -46, "long": 170, "width": 1},
    {"localpath": "Lvl2_1x1/EL_DE/", "filename": "Sat_EL_DE_BA.png", "lat": -47, "long": 170, "width": 1},
]


iceland_data_1Deg = [
    {"localpath": "Lvl2_1x1/AF_FB/", "filename": "Sat_AF_FB_BC.png", "lat": 63, "long": -23, "width": 1},
    {"localpath": "Lvl2_1x1/AF_FB/", "filename": "Sat_AF_FB_BD.png", "lat": 63, "long": -22, "width": 1},
    {"localpath": "Lvl2_1x1/AF_FB/", "filename": "Sat_AF_FB_BE.png", "lat": 63, "long": -21, "width": 1},
    {"localpath": "Lvl2_1x1/AF_FB/", "filename": "Sat_AF_FB_AA.png", "lat": 64, "long": -25, "width": 1},
    {"localpath": "Lvl2_1x1/AF_FB/", "filename": "Sat_AF_FB_AB.png", "lat": 64, "long": -24, "width": 1},
    {"localpath": "Lvl2_1x1/AF_FB/", "filename": "Sat_AF_FB_AC.png", "lat": 64, "long": -23, "width": 1},
    {"localpath": "Lvl2_1x1/AF_FB/", "filename": "Sat_AF_FB_AD.png", "lat": 64, "long": -22, "width": 1},
    {"localpath": "Lvl2_1x1/AF_FB/", "filename": "Sat_AF_FB_AE.png", "lat": 64, "long": -21, "width": 1},

    {"localpath": "Lvl2_1x1/AF_EB/", "filename": "Sat_AF_EB_EA.png", "lat": 65, "long": -25, "width": 1},
    {"localpath": "Lvl2_1x1/AF_EB/", "filename": "Sat_AF_EB_EB.png", "lat": 65, "long": -24, "width": 1},
    {"localpath": "Lvl2_1x1/AF_EB/", "filename": "Sat_AF_EB_EC.png", "lat": 65, "long": -23, "width": 1},
    {"localpath": "Lvl2_1x1/AF_EB/", "filename": "Sat_AF_EB_ED.png", "lat": 65, "long": -22, "width": 1},
    {"localpath": "Lvl2_1x1/AF_EB/", "filename": "Sat_AF_EB_EE.png", "lat": 65, "long": -21, "width": 1},

    {"localpath": "Lvl2_1x1/AF_EB/", "filename": "Sat_AF_EB_DA.png", "lat": 66, "long": -25, "width": 1},
    {"localpath": "Lvl2_1x1/AF_EB/", "filename": "Sat_AF_EB_DB.png", "lat": 66, "long": -24, "width": 1},
    {"localpath": "Lvl2_1x1/AF_EB/", "filename": "Sat_AF_EB_DC.png", "lat": 66, "long": -23, "width": 1},
    {"localpath": "Lvl2_1x1/AF_EB/", "filename": "Sat_AF_EB_DD.png", "lat": 66, "long": -22, "width": 1},
    {"localpath": "Lvl2_1x1/AF_EB/", "filename": "Sat_AF_EB_DE.png", "lat": 66, "long": -21, "width": 1},

    {"localpath": "Lvl2_1x1/AF_FC/", "filename": "Sat_AF_FC_AA.png", "lat": 64, "long": -20, "width": 1},
    {"localpath": "Lvl2_1x1/AF_FC/", "filename": "Sat_AF_FC_AB.png", "lat": 64, "long": -19, "width": 1},
    {"localpath": "Lvl2_1x1/AF_FC/", "filename": "Sat_AF_FC_AC.png", "lat": 64, "long": -18, "width": 1},
    {"localpath": "Lvl2_1x1/AF_FC/", "filename": "Sat_AF_FC_AD.png", "lat": 64, "long": -17, "width": 1},
    {"localpath": "Lvl2_1x1/AF_FC/", "filename": "Sat_AF_FC_AE.png", "lat": 64, "long": -16, "width": 1},

    {"localpath": "Lvl2_1x1/AF_FC/", "filename": "Sat_AF_FC_BA.png", "lat": 63, "long": -20, "width": 1},
    {"localpath": "Lvl2_1x1/AF_FC/", "filename": "Sat_AF_FC_BB.png", "lat": 63, "long": -19, "width": 1},
    {"localpath": "Lvl2_1x1/AF_FC/", "filename": "Sat_AF_FC_BC.png", "lat": 63, "long": -18, "width": 1},
    {"localpath": "Lvl2_1x1/AF_FC/", "filename": "Sat_AF_FC_BD.png", "lat": 63, "long": -17, "width": 1},
    {"localpath": "Lvl2_1x1/AF_FC/", "filename": "Sat_AF_FC_BE.png", "lat": 63, "long": -16, "width": 1},


]

In [21]:
australia_data_5Deg = [
    {"localpath": "Lvl1_5x5/DJ/", "filename": "Sat_DJ_DE.png", "lat": -20, "long": 110, "width": 5},
    {"localpath": "Lvl1_5x5/DJ/", "filename": "Sat_DJ_DF.png", "lat": -20, "long": 115, "width": 5},

    {"localpath": "Lvl1_5x5/DJ/", "filename": "Sat_DJ_EE.png", "lat": -25, "long": 110, "width": 5},
    {"localpath": "Lvl1_5x5/DJ/", "filename": "Sat_DJ_EF.png", "lat": -25, "long": 115, "width": 5},

    {"localpath": "Lvl1_5x5/DJ/", "filename": "Sat_DJ_FE.png", "lat": -30, "long": 110, "width": 5},
    {"localpath": "Lvl1_5x5/DJ/", "filename": "Sat_DJ_FF.png", "lat": -30, "long": 115, "width": 5},


    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_BA.png", "lat": -10, "long": 120, "width": 5},
    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_BB.png", "lat": -10, "long": 125, "width": 5},
    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_BC.png", "lat": -10, "long": 130, "width": 5},
    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_BD.png", "lat": -10, "long": 135, "width": 5},
    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_BE.png", "lat": -10, "long": 140, "width": 5},
    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_BF.png", "lat": -10, "long": 145, "width": 5},

    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_CA.png", "lat": -15, "long": 120, "width": 5},
    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_CB.png", "lat": -15, "long": 125, "width": 5},
    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_CC.png", "lat": -15, "long": 130, "width": 5},
    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_CD.png", "lat": -15, "long": 135, "width": 5},
    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_CE.png", "lat": -15, "long": 140, "width": 5},
    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_CF.png", "lat": -15, "long": 145, "width": 5},

    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_DA.png", "lat": -20, "long": 120, "width": 5},
    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_DB.png", "lat": -20, "long": 125, "width": 5},
    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_DC.png", "lat": -20, "long": 130, "width": 5},
    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_DD.png", "lat": -20, "long": 135, "width": 5},
    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_DE.png", "lat": -20, "long": 140, "width": 5},
    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_DF.png", "lat": -20, "long": 145, "width": 5},

    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_EA.png", "lat": -25, "long": 120, "width": 5},
    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_EB.png", "lat": -25, "long": 125, "width": 5},
    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_EC.png", "lat": -25, "long": 130, "width": 5},
    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_ED.png", "lat": -25, "long": 135, "width": 5},
    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_EE.png", "lat": -25, "long": 140, "width": 5},
    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_EF.png", "lat": -25, "long": 145, "width": 5},

    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_FA.png", "lat": -30, "long": 120, "width": 5},
    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_FB.png", "lat": -30, "long": 125, "width": 5},
    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_FC.png", "lat": -30, "long": 130, "width": 5},
    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_FD.png", "lat": -30, "long": 135, "width": 5},
    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_FE.png", "lat": -30, "long": 140, "width": 5},
    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_FF.png", "lat": -30, "long": 145, "width": 5},


    {"localpath": "Lvl1_5x5/DL/", "filename": "Sat_DL_EA.png", "lat": -25, "long": 150, "width": 5},
    {"localpath": "Lvl1_5x5/DL/", "filename": "Sat_DL_FA.png", "lat": -30, "long": 150, "width": 5},

    {"localpath": "Lvl1_5x5/EJ/", "filename": "Sat_EJ_AE.png", "lat": -35, "long": 110, "width": 5},
    {"localpath": "Lvl1_5x5/EJ/", "filename": "Sat_EJ_AF.png", "lat": -35, "long": 115, "width": 5},

    {"localpath": "Lvl1_5x5/EK/", "filename": "Sat_EK_AA.png", "lat": -35, "long": 120, "width": 5},
    {"localpath": "Lvl1_5x5/EK/", "filename": "Sat_EK_AB.png", "lat": -35, "long": 125, "width": 5},
    {"localpath": "Lvl1_5x5/EK/", "filename": "Sat_EK_AC.png", "lat": -35, "long": 130, "width": 5},
    {"localpath": "Lvl1_5x5/EK/", "filename": "Sat_EK_AD.png", "lat": -35, "long": 135, "width": 5},
    {"localpath": "Lvl1_5x5/EK/", "filename": "Sat_EK_AE.png", "lat": -35, "long": 140, "width": 5},
    {"localpath": "Lvl1_5x5/EK/", "filename": "Sat_EK_AF.png", "lat": -35, "long": 145, "width": 5},

    {"localpath": "Lvl1_5x5/EK/", "filename": "Sat_EK_BD.png", "lat": -40, "long": 135, "width": 5},
    {"localpath": "Lvl1_5x5/EK/", "filename": "Sat_EK_BE.png", "lat": -40, "long": 140, "width": 5},
    {"localpath": "Lvl1_5x5/EK/", "filename": "Sat_EK_BF.png", "lat": -40, "long": 145, "width": 5},

    {"localpath": "Lvl1_5x5/EK/", "filename": "Sat_EK_CE.png", "lat": -45, "long": 140, "width": 5},
    {"localpath": "Lvl1_5x5/EK/", "filename": "Sat_EK_CF.png", "lat": -45, "long": 145, "width": 5},


    {"localpath": "Lvl1_5x5/EL/", "filename": "Sat_EL_AA.png", "lat": -35, "long": 150, "width": 5},
    {"localpath": "Lvl1_5x5/EL/", "filename": "Sat_EL_BA.png", "lat": -40, "long": 150, "width": 5},


]

In [43]:
australia_data_5Deg_2 = [
    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_DA_2.png", "lat": -20, "long": 120, "width": 5},
    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_FD_2.png", "lat": -30, "long": 135, "width": 5},
]

iberia_data_5Deg = [
    {"localpath": "Lvl1_5x5/BF/", "filename": "Sat_BF_DE.png", "lat": 40, "long": -10, "width": 5},
    {"localpath": "Lvl1_5x5/BF/", "filename": "Sat_BF_DF.png", "lat": 40, "long":  -5, "width": 5},
    {"localpath": "Lvl1_5x5/BG/", "filename": "Sat_BG_DA.png", "lat": 40, "long":   0, "width": 5},
    {"localpath": "Lvl1_5x5/BF/", "filename": "Sat_BF_EE.png", "lat": 35, "long": -10, "width": 5},
    {"localpath": "Lvl1_5x5/BF/", "filename": "Sat_BF_EF.png", "lat": 35, "long":  -5, "width": 5},
    {"localpath": "Lvl1_5x5/BG/", "filename": "Sat_BG_EA.png", "lat": 35, "long":   0, "width": 5},
]

iberia_data_1Deg = [
    {"localpath": "Lvl2_1x1/BF_DE/", "filename": "Sat_BF_DE_AA.png", "lat": 44, "long": -10, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DE/", "filename": "Sat_BF_DE_AB.png", "lat": 44, "long":  -9, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DE/", "filename": "Sat_BF_DE_AC.png", "lat": 44, "long":  -8, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DE/", "filename": "Sat_BF_DE_AD.png", "lat": 44, "long":  -7, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DE/", "filename": "Sat_BF_DE_AE.png", "lat": 44, "long":  -6, "width": 1},

    {"localpath": "Lvl2_1x1/BF_DE/", "filename": "Sat_BF_DE_BA.png", "lat": 43, "long": -10, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DE/", "filename": "Sat_BF_DE_BB.png", "lat": 43, "long":  -9, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DE/", "filename": "Sat_BF_DE_BC.png", "lat": 43, "long":  -8, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DE/", "filename": "Sat_BF_DE_BD.png", "lat": 43, "long":  -7, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DE/", "filename": "Sat_BF_DE_BE.png", "lat": 43, "long":  -6, "width": 1},

    {"localpath": "Lvl2_1x1/BF_DE/", "filename": "Sat_BF_DE_CA.png", "lat": 42, "long": -10, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DE/", "filename": "Sat_BF_DE_CB.png", "lat": 42, "long":  -9, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DE/", "filename": "Sat_BF_DE_CC.png", "lat": 42, "long":  -8, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DE/", "filename": "Sat_BF_DE_CD.png", "lat": 42, "long":  -7, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DE/", "filename": "Sat_BF_DE_CE.png", "lat": 42, "long":  -6, "width": 1},

    {"localpath": "Lvl2_1x1/BF_DE/", "filename": "Sat_BF_DE_DA.png", "lat": 41, "long": -10, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DE/", "filename": "Sat_BF_DE_DB.png", "lat": 41, "long":  -9, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DE/", "filename": "Sat_BF_DE_DC.png", "lat": 41, "long":  -8, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DE/", "filename": "Sat_BF_DE_DD.png", "lat": 41, "long":  -7, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DE/", "filename": "Sat_BF_DE_DE.png", "lat": 41, "long":  -6, "width": 1},

    {"localpath": "Lvl2_1x1/BF_DE/", "filename": "Sat_BF_DE_EA.png", "lat": 40, "long": -10, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DE/", "filename": "Sat_BF_DE_EB.png", "lat": 40, "long":  -9, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DE/", "filename": "Sat_BF_DE_EC.png", "lat": 40, "long":  -8, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DE/", "filename": "Sat_BF_DE_ED.png", "lat": 40, "long":  -7, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DE/", "filename": "Sat_BF_DE_EE.png", "lat": 40, "long":  -6, "width": 1},
]

In [45]:
RootDir = "C:/Util/GlobeLibrary_MapPrep/_WIP/exp/MapLib/"
ensure_dir(RootDir)

brightness = 35

oauth = SetupOAuth(client_id, client_secret)


# Loop over the data structure
for data in iberia_data_1Deg:

    TilesetDir = os.path.join(RootDir, data["localpath"])
    filepath   = os.path.join(TilesetDir, data["filename"])

    ensure_dir(TilesetDir)

    print(filepath)

    if not os.path.exists(filepath):

        print(f"Downloading and enhancing image: {data['filename']}")

        # Call your functions using the metadata
        download_GM(filepath, data["lat"], data["long"], data["width"])
        # change_absolute_brightness(filepath, filepath, brightness)
        #adjust_image_properties(filepath, filepath, brightness_delta=30, contrast_factor=1.5, saturation_factor=1.5)





C:/Util/GlobeLibrary_MapPrep/_WIP/exp/MapLib/Lvl2_1x1/BF_DE/Sat_BF_DE_AA.png
C:/Util/GlobeLibrary_MapPrep/_WIP/exp/MapLib/Lvl2_1x1/BF_DE/Sat_BF_DE_AB.png
C:/Util/GlobeLibrary_MapPrep/_WIP/exp/MapLib/Lvl2_1x1/BF_DE/Sat_BF_DE_AC.png
C:/Util/GlobeLibrary_MapPrep/_WIP/exp/MapLib/Lvl2_1x1/BF_DE/Sat_BF_DE_AD.png
C:/Util/GlobeLibrary_MapPrep/_WIP/exp/MapLib/Lvl2_1x1/BF_DE/Sat_BF_DE_AE.png
C:/Util/GlobeLibrary_MapPrep/_WIP/exp/MapLib/Lvl2_1x1/BF_DE/Sat_BF_DE_BA.png
C:/Util/GlobeLibrary_MapPrep/_WIP/exp/MapLib/Lvl2_1x1/BF_DE/Sat_BF_DE_BB.png
C:/Util/GlobeLibrary_MapPrep/_WIP/exp/MapLib/Lvl2_1x1/BF_DE/Sat_BF_DE_BC.png
C:/Util/GlobeLibrary_MapPrep/_WIP/exp/MapLib/Lvl2_1x1/BF_DE/Sat_BF_DE_BD.png
C:/Util/GlobeLibrary_MapPrep/_WIP/exp/MapLib/Lvl2_1x1/BF_DE/Sat_BF_DE_BE.png
C:/Util/GlobeLibrary_MapPrep/_WIP/exp/MapLib/Lvl2_1x1/BF_DE/Sat_BF_DE_CA.png
C:/Util/GlobeLibrary_MapPrep/_WIP/exp/MapLib/Lvl2_1x1/BF_DE/Sat_BF_DE_CB.png
C:/Util/GlobeLibrary_MapPrep/_WIP/exp/MapLib/Lvl2_1x1/BF_DE/Sat_BF_DE_CC.png